# 07. CLONE DETECTION

1. [x] введение
2. [x] подходы
3. [x] упражнение
3. [x] Ссылки

# 1. Введение

## Что такое клоны?

Клоны — это фрагменты кода, похожие в соответствии с некоторым определением подобия.

В соответствии с этим определением могут быть разные понятия подобия. Они могут быть основаны на тексте, лексической или синтаксической структуре или семантике. Часто возникают в результате "copy & paste".

## Какие бывают?

#### Type-1 (Exact clones)

Идентичные фрагменты кода, за исключением различий в пробелах, комментариях и вёрстке.

#### Type-2 (Renamed/Parameterized)

Идентичные фрагменты кода, за исключением различий в именах идентификаторов и литеральных значениях, в дополнение к различиям клонов типа 1.

#### Type-3 (Near miss clones/Gapped clones)

Синтаксически похожие фрагменты кода, которые содержат добавленные, измененные и/или удаленные операторы кода, в дополнение к различиям клонов типа 1 и типа 2.

#### Type-4 (Semantic clones)

Синтаксически разные фрагменты кода, но семантически схожие с точки зрения функционала.


> Примеры:
> ![types](./res/07_clone_types.png)
>
> Эквивалентные преобразования:
> ![types](./res/07_equivalent_transformations.png)

## Почему это плохо?

Несмотря на то, что клонирование может быть полезным во многих отношениях, оно также может быть вредным при обслуживании и развитии программного обеспечения.

Например, если в каком-то фрагменте кода обнаружена ошибка, то все клоны должны быть проверены на наличие такой же ошибки. Дублированные фрагменты также могут значительно увеличить объем работы при улучшении или адаптации кода.


Кроме того, дедупликация (исключение клонов) --- стандартный процесс при подготовке датасетов.

## Почему эта проблема значимая?

Повторное использование фрагментов кода путем копирования и вставки с незначительной адаптацией или без нее — обычное дело при разработке программного обеспечения.

В результате программные системы часто содержат клоны. Исследования показывают, что значительная часть (от 7% до 23%) кода в типичной программной системе была клонирована.

- Linux: 22.3% ([Sheneamer Kalita, 2016](https://www.ijcaonline.org/archives/volume137/number10/24308-2016908896))
- JDK: 29% ([Kamiya et al, 2002](https://www.cs.drexel.edu/~spiros/teaching/CS675/papers/clone-kamiya.pdf))

70% кода на GitHub состоит из клонов ранее созданных файлов.


#### Карта дублирования кода ([Lopes et al, 2017](http://janvitek.org/pubs/oopsla17b.pdf)):

- ось Y --- количество коммитов на проект
- ось X --- количество файлов в проекте.

Значение на каждой плитки --- это процент дублированных файлов для всех проектов в плитке.

![dejavu](./res/07_dejavu_tiles_1.png)
![dejavu](./res/07_dejavu_tiles_2.png)

# 2. Подходы

- текст
- токены
- вектора
- AST
- графы
- изображения

## Сравнение на основе текста

Например, сравнения на уровне строк.
- 1. нормализуем код
- 2. для каждой строки вычисляем значение некоторой хэш-функции
- 3. визуализируем в виде матрицы: если две строки имеют одинаковое значение хэш-функции, то пишем 1, иначе 0
- 4. выявляем паттеры (например, диагональные последовательности) из единиц, они соответствуют повторяющемуся коду

Другие подходы используют, например, эффективные алгоритмы сравнения строк.

> Пример:
>
> ([Ducasse et al, 1999](https://ieeexplore.ieee.org/document/792593))
> 
> ![](./res/07_clones_text.png)

## Сравнение на основе токенов

Сравнение тоже может происходит на уровне строк. Но вместо сравнения символов используется сравнение токенов.

В случае подходов на основе токенов каждая строка исходного кода делится на токены в соответствии с лексическими правилами интересующего языка программирования. Токены вместе образуют последовательность токенов, используемую для сравнения. Все пробелы (включая разрывы строк и табуляции) и комментарии между токенами удаляются из последовательностей токенов.

Например, если фрагменты кода отличаются только названиями переменных, то разные названия будут соответствовать одному и тому же токены и, поэтому, не помешают при сравнении.

> Пример:
> 
> [CCFinder](https://www.cs.drexel.edu/~spiros/teaching/CS675/papers/clone-kamiya.pdf), [CCFinderX](https://github.com/gpoo/ccfinderx)
>

## Сравнение на основе абстрактных синтаксических деревьев

Например, так:
1. строится абстрактное синтаксическое дерево
2. выделяются поддеревья
3. вычисляются значения некоторой хэш-функции для этих поддеревьев
4. если два поддерева получили одинаково значение, то происходит дополнительное сравнение двух деревьев


![](https://leanovate.github.io/bedcon/talk/abstract_syntax_tree.png)

## Сравнение на основе потока данных

Методы на основе токенов и методы на основе синтаксиса зависят от текстового порядка программы. Если программисты изменяют порядок операторов в скопированном коде, то скопированный код не будет найден.

Тем не менее, порядок не может быть изменен произвольно без изменения смысла программы.
Граф программных зависимостей представляет собой представление программы, которое представляет только зависимость управления и данных между операторами.

![](https://www.researchgate.net/profile/Sergey-Troshin/publication/358740900/figure/fig1/AS:11431281080328895@1661254252291/Example-of-Data-Flow-Graph-from-GraphCodeBERT-Guo-et-al-2021_W640.jpg)

Клоны могут быть идентифицированы как изоморфные подграфы в графе программных зависимостей.

Задача является NP-сложной, используются приближённые алгоритмы.

> Пример.
> Изоморфизм графов
> ![](https://static.javatpoint.com/tutorial/dms/images/graph-isomorphism-in-discrete-mathematics.png)

## Сравнение векторов

Происходит сравнение векторов в метрическом пространстве. Вектора строятся на основе различных метрик или предобученных моделей (BERT, RoBERTa, CodeBERT, GraphCodeBERT, ...).

#### Contrastive learning

Contrastive learning --- это подход в машинном обучении, для выявления похожих и непохожих объектов.
Модель обучается строить эмбеддинги для объектов таким образом, что похожие объекты будут иметь близкие вектора, а разные объекты, соответственно, будут различаться.
Поэтому решение о том, являются ли две программы клонами или нет, определяется расстоянием между двумя векторами.

> Пример. [SimCLR](https://arxiv.org/abs/2002.05709)
>
> ![](./res/07_clones_sim_clr.png)

## Сравнение на основе изображений

[Ragkhitwetsagul et al - A picture is worth a thousand words Code clone detection based on image similarity 2018](https://ieeexplore.ieee.org/document/8327318)

![scheme](./res/07_clone_detection_image_scheme.png)

Подход:

1. 
   - распарсить код и извлечь методы
   - удалить комментарии
   - красиво напечатать
   - преобразовать в html с подсветкой синтаксиса
2. 
   - конвертировать каждый метод в PNG-изображение
   - прочить RGB-изображение
3. 
   - преобразовать изображение в негативное изображение
   - применить фильтры (гауссовый фильтр, например)
4. 
   - сравнить два изображения. Можено использовать **The Earth Mover’s Distance (EMD)** --- метрику, которая рассматривает сравнение изображений как транспортную задачу, находя минимальную стоимость преобразования одного распределения в другое
  
![](./res/07_clone_detection_image_1.png)
![](./res/07_clone_detection_image_2.png)
![](./res/07_clone_detection_image_3.png)

## Evaluation

- [BigCloneBench](https://github.com/clonebench/BigCloneBench) --- включает более 8 миллионов клонов из 25000 Java-репозиториев
- [BigCloneEval](https://github.com/jeffsvajlenko/BigCloneEval) --- фреймворк для тестирования разных методов по детекции клонов

# 3. Упражнение

Реализовать поиск клонов с помощью модели CodeBERT. Проверить на датасете BigCloneBench.

См. задача 4

# 4. Ссылки
- Allamanis - The adverse effects of code duplication in machine learning models of code
- Baker - On finding duplication and near-duplication in large software systems
- Bogomolov et al - Sosed A tool for finding similar software projects
- [Code code analysis](https://link.springer.com/book/10.1007/978-981-16-1927-4)
- Ducasse et al - A language independent approach for detecting duplicated code
- Gupta Gupta - Literature survey of clone detection techniques
- Huang et al - Code clone detection based on event embedding and event dependency
- Ivanov et al - AntiCopyPaster Extracting code duplicates as soon as they are introduced in the IDE
- Khajezade et al - Evaluating few shot and contrastive learning methods for code clone detection
- Kim et al - Revisiting binary code similarity analysis using interpretable feature engineering and lessons learned
- Koschke - Survey of research on software clones
- Lopes et al - DejaVu A map of code duplicates on GitHub
- Nadim et al - Evaluating the performance of clone detection tools in detecting cloned co-change candidates
- Ragkhitwetsagul - Code similarity and clone search in large-scale source code data
- Ragkhitwetsagul et al - A picture is worth a thousand words Code clone detection based on image similarity
- Rahman et al - Clone detection on large Scala codebases
- Roy et al - Comparison and evaluation of code clone detection techniques and tools
- Saini et al - Oreo Detection of clones in the twilight zone
- Sheneamer Kalita - A survey of software clone detection techniques
- [Svajlenko Roy - A survey on the evaluation of clone detection performance and benchmarking](https://arxiv.org/abs/2006.15682)
- Svajlenko Roy - Efficiently measuring an accurate and generalized clone detection precision using clone clustering
- White et al - Deep learning code fragments for code clone detection
- Yahay Kim - Cross-languages source code clone detection using deep learning with InferCode
- Zhang et al - Challenging machine learning-based clone detectors via semantic-preserving code transformations
- Zhang et al - The development and prospect of code clone
- Zubkov et al - Evaluation of contrastive learning with various code representations for code clone detection 2022